In [22]:
include("../code/headers.jl")

In [17]:
m = factorial(5)*11
R = 0.8/3
r = 1 - 3R
n = round(Int, 3m/(2+r))
Λ = [0,1-r,r]
K = [0,0,1]
nedges = 3m
lambda,rho = nodes2edges(Λ,K)
fg = ldpc_graphGF2(n,m,nedges,lambda,rho)
breduction!(fg, 10)
lm = LossyModel(fg)

Lossy compression model:
 - Factor Graph with n=1800 variables and m=1310 factors defined on GF(2)
 - Inverse temperatures β₁=1.0 for checks and β₂=1.0 for overlap


In [21]:
depths, var2fact, vars_order = lr(fg)
all(depths .!= 0)

true

In [23]:
seaweed(fg, 1)

LoadError: StackOverflowError:

In [6]:
cw = falses(n)
dist = 0.3
s = [rand()<dist for _ in 1:n]
mean(s)

0.3022222222222222

In [16]:
depths=lr(fg)[1]
isincore= (depths .== 0)

1800-element BitArray{1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [14]:
mc_move = MetropSmallJumps()
adapt_to_model!(mc_move, lm)
to_flip, newvals = propose(mc_move, lm, MersenneTwister(123))

LoadError: StackOverflowError: